# Helper functions

> Helper: Module containing helper functions for mlflow etc

In [1]:
#| default_exp helper

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| hide
import nbdev; nbdev.nbdev_export()

In [3]:
#| export
from loguru import logger
import os
from pathlib import Path
from fastcore.basics import patch_to, patch

import pandas as pd

from typing import Union, List

from datetime import datetime

## DataFrame

In [26]:
#| export
def view_df(df: "pd.DataFrame", # Pandas DataFrame to be viewed
            min_rows: int = 60, # minimum row 
            max_colswidth: int = 500, # maximum width of the column
            max_cols: int = None # maximum columns
           ):
    "View dataframe in full columns in Jupyter! If `max_cols==None`, it will show the full column."
    
    with pd.option_context('display.max_columns', max_cols, 'display.min_rows', min_rows, 'display.max_colwidth', max_colswidth):
        display(df)

## Others

In [8]:
#| export
def get_now():
    " Returns the time now in 'yyyy-mm-dd_HHMMSS"
    return datetime.now().strftime("%Y%m%d_%H%M%S")

In [9]:
get_now()

'20231028_021425'

In [10]:
#| export
def get_today(fmt="%Y-%m-%d"):
    " Returns today's date (default format: 'yyyy-mm-dd')"
    return datetime.today().strftime(fmt)

In [27]:
#| hide
import nbdev; nbdev.nbdev_export()